In [1]:
import qstrader

In [2]:
import os

import pandas as pd
import pytz

# alpha signal
from qstrader.alpha_model.fixed_signals import FixedSignalsAlphaModel

# asset types and universe
from qstrader.asset.equity import Equity
from qstrader.asset.universe.static import StaticUniverse

# data reader and provider
from qstrader.data.backtest_data_handler import BacktestDataHandler
from qstrader.data.daily_bar_csv import CSVDailyBarDataSource

# stats generator
from qstrader.statistics.tearsheet import TearsheetStatistics

# main thing
from qstrader.trading.backtest import BacktestTradingSession




In [3]:
start_dt = pd.Timestamp('2003-09-30 09:30:00', tz='America/New_York')
end_dt = pd.Timestamp('2019-12-31 16:00:00', tz='America/New_York')

# Construct the symbols and assets necessary for the backtest
strategy_symbols = ['SPY', 'AGG']
strategy_assets = ['EQ:%s' % symbol for symbol in strategy_symbols]
strategy_universe = StaticUniverse(strategy_assets)

In [4]:
os.environ["QSTRADER_CSV_DATA_DIR"] = '/Users/snk/pcode/archive/qstrader/data'

In [5]:
# To avoid loading all CSV files in the directory, set the
# data source to load only those provided symbols
csv_dir = os.environ.get('QSTRADER_CSV_DATA_DIR', '.')
data_source = CSVDailyBarDataSource(csv_dir , Equity, csv_symbols=strategy_symbols)
data_handler = BacktestDataHandler(strategy_universe, data_sources=[data_source])

Loading CSV files into DataFrames...
Loading CSV file for symbol 'EQ:SPY'...
Loading CSV file for symbol 'EQ:AGG'...
Loading pricing in CSV files...
Loading CSV file for symbol 'EQ:SPY'...
Loading CSV file for symbol 'EQ:AGG'...


In [6]:
dt = pd.Timestamp('2000-03-17 16:00:00', tz='America/New_York')


In [7]:
dt

Timestamp('2000-03-17 16:00:00-0500', tz='America/New_York')

In [8]:

bid_ask_df = data_source.asset_bid_ask_frames['EQ:SPY']
bid_ask_df

bid_ask_df.index.get_indexer([dt])
bid_series = bid_ask_df.iloc[bid_ask_df.index.get_indexer([dt], method='pad')]['Bid']
bid_ask_df.index.get_indexer([dt], method='pad')

array([105])

In [9]:
div_df = data_source.dividend_frames['EQ:SPY']

In [10]:
data_source.get_ask(dt, 'EQ:SPY')

np.float64(146.9375)

In [11]:
data_source.get_dividend(dt, 'EQ:SPY')

np.float64(0.371)

In [12]:
snp_div = div_df['Dividends'].pipe(lambda x: x[x!=0])#.to_csv('snp_div.csv')

In [6]:
# Construct an Alpha Model that simply provides
# static allocations to a universe of assets
# In this case 60% SPY ETF, 40% AGG ETF,
# rebalanced at the end of each month
strategy_alpha_model = FixedSignalsAlphaModel({'EQ:SPY': 0.6, 'EQ:AGG': 0.4})
strategy_backtest = BacktestTradingSession(
    start_dt,
    end_dt,
    strategy_universe,
    strategy_alpha_model,
    rebalance='end_of_month',
    long_only=True,
    cash_buffer_percentage=0.01,
    data_handler=data_handler,
    process_dividends=True,
    reinvest_dividends=True,
    

)
strategy_backtest.run(results = False)

Initialising simulated broker "Backtest Simulated Broker Account"...
(2003-09-30 09:30:00-04:00) - portfolio creation: Portfolio "000001" created at broker "Backtest Simulated Broker Account"
(2003-09-30 09:30:00-04:00) - subscription: 1000000.00 subscribed to portfolio "000001"
Beginning backtest simulation...
(2003-09-30 09:30:00-04:00) - market_open
(2003-09-30 16:00:00-04:00) - market_close
(2003-09-30 16:00:00-04:00) - trading logic and rebalance
(2003-09-30 16:00:00-04:00) - target weights: {'EQ:AGG': 0.4, 'EQ:SPY': 0.6}
(2003-09-30 16:00:00-04:00) - submitted order: EQ:AGG, qty: 3855
(2003-09-30 16:00:00-04:00) - submitted order: EQ:SPY, qty: 5942
(2003-10-01 09:30:00-04:00) - market_open
(2003-10-01 09:30:00-04:00) - executed order: EQ:AGG, qty: 3855, price: 102.64, consideration: 395677.00, commission: 0.00, total: 395677.00
(2003-10-01 09:30:00-04:00) - executed order: EQ:SPY, qty: 5942, price: 100.24, consideration: 595626.00, commission: 0.00, total: 595626.00
(2003-10-01 1

In [15]:
dt

Timestamp('2000-03-17 16:00:00-0500', tz='America/New_York')

In [ ]:
pd.DataFrame(strategy_backtest.stats['dividends']).pipe(lambda x: x[x['total_cash_dividend']!= 0])

In [ ]:
strategy_backtest.data_handler.get_asset_dividend(dt, 'EQ:SPY')

In [ ]:
stats = strategy_backtest.get_stats_dataframe()

In [ ]:
stats['equity_curve']

In [8]:
jstats = strategy_backtest.stats

In [9]:
for div in jstats['dividends']:
    if div['date'] == pd.Timestamp('2003-12-01 16:00:00', tz='America/New_York'):
        break 

In [10]:
[div,div]

[{'date': Timestamp('2003-12-01 16:00:00-0500', tz='America/New_York'),
  'dividends': [{'asset': 'EQ:AGG',
    'dividend': np.float64(0.272),
    'quantity': 4028,
    'cash_dividend': np.float64(1095.616),
    'reinvest_price': np.float64(101.16999816894533),
    'reinvested_quantity': 10}],
  'total_cash_dividend': np.float64(1095.616),
  'total_reinvested_quantity': 10,
  'event': 'market_close'},
 {'date': Timestamp('2003-12-01 16:00:00-0500', tz='America/New_York'),
  'dividends': [{'asset': 'EQ:AGG',
    'dividend': np.float64(0.272),
    'quantity': 4028,
    'cash_dividend': np.float64(1095.616),
    'reinvest_price': np.float64(101.16999816894533),
    'reinvested_quantity': 10}],
  'total_cash_dividend': np.float64(1095.616),
  'total_reinvested_quantity': 10,
  'event': 'market_close'}]

In [9]:
jstats['dividends'][0]

{'date': Timestamp('2003-09-30 16:00:00-0400', tz='America/New_York'),
 'dividends': [],
 'total_cash_dividend': 0.0,
 'total_reinvested_quantity': 0,
 'event': 'market_close'}

In [10]:
pd.DataFrame(jstats['dividends'])

,date,dividends,total_cash_dividend,total_reinvested_quantity,event
0,2003-09-30 16:00:00-04:00,[],0.0,0,market_close
1,2003-10-01 16:00:00-04:00,[],0.0,0,market_close
2,2003-10-02 16:00:00-04:00,[],0.0,0,market_close
3,2003-10-03 16:00:00-04:00,[],0.0,0,market_close
4,2003-10-06 16:00:00-04:00,[],0.0,0,market_close
...,...,...,...,...,...
4236,2019-12-25 16:00:00-05:00,[],0.0,0,market_close
4237,2019-12-26 16:00:00-05:00,[],0.0,0,market_close
4238,2019-12-27 16:00:00-05:00,[],0.0,0,market_close
4239,2019-12-30 16:00:00-05:00,[],0.0,0,market_close


In [11]:
import json
from pandas import Timestamp


def convert_timestamps(obj):
    if isinstance(obj, dict):
        return {k: convert_timestamps(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_timestamps(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_timestamps(item) for item in obj)  # Convert tuple elements
    elif isinstance(obj, Timestamp):
        return obj.isoformat()
    elif pd.isna(obj):  # Check for NaN values
        return 0
    return obj

# Convert and save
converted_data = convert_timestamps(jstats.copy())

In [12]:
converted_data['current_portfolio']

[{'date': '2003-09-30T16:00:00-04:00',
  'event': 'market_close',
  'portfolio': {}},
 {'date': '2003-10-31T16:00:00-05:00',
  'event': 'market_close',
  'portfolio': {'EQ:AGG': {'quantity': 3855,
    'market_value': np.float64(392207.6917648315),
    'unrealised_pnl': np.float64(-3469.505882263238),
    'realised_pnl': 0.0,
    'total_pnl': np.float64(-3469.505882263238)},
   'EQ:SPY': {'quantity': 5942,
    'market_value': np.float64(625692.6181335448),
    'unrealised_pnl': np.float64(30066.550827026367),
    'realised_pnl': 0.0,
    'total_pnl': np.float64(30066.550827026367)}}},
 {'date': '2003-11-28T16:00:00-05:00',
  'event': 'market_close',
  'portfolio': {'EQ:AGG': {'quantity': 4009,
    'market_value': np.float64(407795.4848937988),
    'unrealised_pnl': np.float64(-3488.212966918925),
    'realised_pnl': 0.0,
    'total_pnl': np.float64(-3488.212966918925)},
   'EQ:SPY': {'quantity': 5791.0,
    'market_value': np.float64(616451.9323272706),
    'unrealised_pnl': np.float64(

In [ ]:
converted_data.keys()

In [ ]:
keys_to_keep = ['dates', 'events', 'alpha_weights', 'risk_weights', 'optimised_weights','target_portfolio', 'current_portfolio','rebalance_orders', 'executed_orders','equity_curve']

In [13]:
pd.DataFrame(converted_data['dividends'])

,date,dividends,total_cash_dividend,total_reinvested_quantity,event
0,2003-09-30T16:00:00-04:00,[],0.0,0,market_close
1,2003-10-01T16:00:00-04:00,[],0.0,0,market_close
2,2003-10-02T16:00:00-04:00,[],0.0,0,market_close
3,2003-10-03T16:00:00-04:00,[],0.0,0,market_close
4,2003-10-06T16:00:00-04:00,[],0.0,0,market_close
...,...,...,...,...,...
4236,2019-12-25T16:00:00-05:00,[],0.0,0,market_close
4237,2019-12-26T16:00:00-05:00,[],0.0,0,market_close
4238,2019-12-27T16:00:00-05:00,[],0.0,0,market_close
4239,2019-12-30T16:00:00-05:00,[],0.0,0,market_close


In [ ]:
[{'asset': 'EQ:AGG', 'dividend': np.float64(0.291), 'quantity': 4038, 'cash_dividend': np.float64(1175.058), 'reinvest_price': 0, 'reinvested_quantity': 0}]

In [14]:
with open('6040stats.json', 'w') as f:
    json.dump(converted_data, f)

In [ ]:
tearsheet = TearsheetStatistics(
            strategy_equity=strategy_backtest.get_equity_curve(),
            benchmark_equity=benchmark_backtest.get_equity_curve(),
            title=strat_title
        )

In [ ]:
tearsheet = TearsheetStatistics(
    strategy_equity=strategy_backtest.get_equity_curve(),
    benchmark_equity=benchmark_backtest.get_equity_curve(),
    title='60/40 US Equities/Bonds'
)
# tearsheet.plot_results()

In [ ]:
tearsheet

In [ ]:
# Construct benchmark assets (buy & hold SPY)
benchmark_assets = ['EQ:SPY']
benchmark_universe = StaticUniverse(benchmark_assets)

# Construct a benchmark Alpha Model that provides
# 100% static allocation to the SPY ETF, with no rebalance
benchmark_alpha_model = FixedSignalsAlphaModel({'EQ:SPY': 1.0})
benchmark_backtest = BacktestTradingSession(
    start_dt,
    end_dt,
    benchmark_universe,
    benchmark_alpha_model,
    rebalance='buy_and_hold',
    long_only=True,
    cash_buffer_percentage=0.01,
    data_handler=data_handler
)
benchmark_backtest.run()

# Performance Output
tearsheet = TearsheetStatistics(
    strategy_equity=strategy_backtest.get_equity_curve(),
    benchmark_equity=benchmark_backtest.get_equity_curve(),
    title='60/40 US Equities/Bonds'
)
tearsheet.plot_results()